In [1]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import data as dt
import functions as fn
from collections import Counter
import ta
import yfinance as yf

In [2]:
BTCUSDT15m = dt.BTCUSDT15m
BTCUSDT15m.set_index("Open Time", inplace=True)

In [3]:
# SRSI
BTCUSDT15m1 = pd.concat([BTCUSDT15m, pta.stochrsi(BTCUSDT15m.Close, 14)], axis = 1)
BTCUSDT15m1.rename(columns = {'STOCHRSIk_14_14_3_3':'K', 'STOCHRSId_14_14_3_3':'D'}, inplace = True)

In [4]:
# EMA
for i in (8, 14, 50):
    BTCUSDT15m1['EMA_'+str(i)] = pta.ema(BTCUSDT15m1.Close, i)

In [5]:
# ATR
BTCUSDT15m1['ATR'] =pta.atr(BTCUSDT15m1.High, BTCUSDT15m1.Low, BTCUSDT15m1.Close)

In [6]:
BTCUSDT15m1.dropna(inplace=True)

In [7]:
BTCUSDT15m1.head(2)

,Open,High,Low,Close,Volume,K,D,EMA_8,EMA_14,EMA_50,ATR
Open Time,,,,,,,,,,,
2018-01-01 12:15:00,13247.38,13247.38,13056.23,13146.15,67.242759,36.729872,28.624030,13268.291764,13349.527119,13490.117800,155.891434
2018-01-01 12:30:00,13146.81,13245.55,13146.28,13200.01,66.739212,38.382290,35.686151,13253.118039,13329.591503,13478.741024,151.754605


In [8]:
def check_cross(df):
    return (df['K'] > df['D']) & (df['K'] > df['D']).diff()

In [9]:
BTCUSDT15m1['KD_Cross'] = check_cross(BTCUSDT15m1)
BTCUSDT15m1.dropna(inplace=True)

In [10]:
BTCUSDT15m1['TP'] = BTCUSDT15m1.Close+(BTCUSDT15m1.ATR*2)

In [11]:
BTCUSDT15m1['SL'] = BTCUSDT15m1.Close-(BTCUSDT15m1.ATR*3)

In [12]:
BTCUSDT15m1.head(2)

,Open,High,Low,Close,Volume,K,D,EMA_8,EMA_14,EMA_50,ATR,KD_Cross,TP,SL
Open Time,,,,,,,,,,,,,,
2018-01-01 12:15:00,13247.38,13247.38,13056.23,13146.15,67.242759,36.729872,28.624030,13268.291764,13349.527119,13490.117800,155.891434,False,13457.932867,12678.475699
2018-01-01 12:30:00,13146.81,13245.55,13146.28,13200.01,66.739212,38.382290,35.686151,13253.118039,13329.591503,13478.741024,151.754605,False,13503.519210,12744.746185


In [13]:
BTCUSDT15m1['Buy_signal'] = np.where((BTCUSDT15m1.KD_Cross) &
                                     (BTCUSDT15m1.Close > BTCUSDT15m1.EMA_8) &
                                     (BTCUSDT15m1.EMA_8 > BTCUSDT15m1.EMA_14) &
                                     (BTCUSDT15m1.EMA_14 > BTCUSDT15m1.EMA_50), 1, 0)

In [14]:
selldates = []
outcome = []
for i in range(len(BTCUSDT15m1)):
    if BTCUSDT15m1.Buy_signal.iloc[i]:
        k = 1
        SL = BTCUSDT15m1.SL.iloc[i]
        TP = BTCUSDT15m1.TP.iloc[i]
        in_position = True
        while in_position:
            looping_high = BTCUSDT15m1.High.iloc[i + k]
            looping_low = BTCUSDT15m1.Low.iloc[i+k]
            if looping_high >= TP:
                selldates.append(BTCUSDT15m1.iloc[i + k].name)
                outcome.append('TP')
                in_position = False
            elif looping_low <= SL:
                selldates.append(BTCUSDT15m1.iloc[i + k].name)
                outcome.append('SL')
                in_position = False
            k +=1

In [15]:
BTCUSDT15m1.loc[selldates, 'Sell_signal'] = 1

In [16]:
BTCUSDT15m1.loc[selldates, 'Outcome'] = outcome

In [17]:
BTCUSDT15m1.Sell_signal = BTCUSDT15m1.Sell_signal.fillna(0).astype(int)

In [18]:
mask = BTCUSDT15m1[(BTCUSDT15m1.Buy_signal == 1) | (BTCUSDT15m1.Sell_signal == 1)]

In [19]:
mask2 = mask[(mask.Buy_signal.diff() == 1) | (mask.Sell_signal.diff() == 1)]

In [20]:
RES = mask2.Outcome.value_counts()
RES

TP    1819
SL    1174
Name: Outcome, dtype: int64

In [21]:
win_rate = RES[0]/(RES[0]+RES[1])
win_rate

0.6077514199799532

In [22]:
loss_rate = RES[1]/(RES[0]+RES[1])
loss_rate

0.39224858002004676

In [23]:
BTCUSDT15m1[['Buy_signal', 'Sell_signal']] = 0
BTCUSDT15m1['Outcome'] = np.NaN

In [24]:
BTCUSDT15m1.loc[mask2.index.values, 'Buy_signal'] = mask2['Buy_signal']
BTCUSDT15m1.loc[mask2.index.values, 'Sell_signal'] = mask2['Sell_signal']
BTCUSDT15m1.loc[mask2.index.values, 'Outcome'] = mask2['Outcome']

In [25]:
train1 = BTCUSDT15m1['2018-01-01 00:00:00': '2018-01-05 00:00:00']

In [26]:
train1.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'K', 'D', 'EMA_8', 'EMA_14',
       'EMA_50', 'ATR', 'KD_Cross', 'TP', 'SL', 'Buy_signal', 'Sell_signal',
       'Outcome'],
      dtype='object')

In [27]:
train1[['Close', 'Buy_signal', 'Sell_signal', 'Outcome']]['2018-01-02 09:00:00': '2018-01-02 16:00:00']

,Close,Buy_signal,Sell_signal,Outcome
Open Time,,,,
2018-01-02 09:00:00,13480.00,0,0,NaN
2018-01-02 09:15:00,13451.06,0,0,NaN
2018-01-02 09:30:00,13450.49,0,0,NaN
2018-01-02 09:45:00,13452.00,0,0,NaN
2018-01-02 10:00:00,13539.87,1,0,NaN
2018-01-02 10:15:00,13520.01,0,0,NaN
2018-01-02 10:30:00,13559.99,0,0,NaN
2018-01-02 10:45:00,13560.00,0,0,NaN
2018-01-02 11:00:00,13500.01,0,0,NaN


In [28]:
def pointpos(x, type):
    if type == 'Buy':
        if x['Buy_signal']== 1:
            return x['Close']
        else:
            return np.nan
    elif type == 'Sell':
        if x['Sell_signal']== 1:
            return x['Close']
        else:
            return np.nan

train1.loc[:,'buys'] = train1.apply(lambda row: pointpos(row, 'Buy'), axis=1)
train1.loc[:,'sells'] = train1.apply(lambda row: pointpos(row, 'Sell'), axis=1)

C:\Users\rhg22\AppData\Local\Temp\ipykernel_6696\1929773315.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1.loc[:,'buys'] = train1.apply(lambda row: pointpos(row, 'Buy'), axis=1)
C:\Users\rhg22\AppData\Local\Temp\ipykernel_6696\1929773315.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train1.loc[:,'sells'] = train1.apply(lambda row: pointpos(row, 'Sell'), axis=1)


In [29]:
fig_001 = go.Figure(
    data=[
        go.Candlestick(
            x=train1.index,
            open=train1["Open"],
            high=train1["High"],
            low=train1["Low"],
            close=train1["Close"]
        ),
        go.Scatter(x=train1.index, y=train1['K'], line=dict(color='blue', width=1.5),mode="lines", name="K", yaxis="y2"),
        go.Scatter(x=train1.index, y=train1['D'], line=dict(color='red', width=1.5),mode="lines", name="D", yaxis="y2"),
        go.Scatter(x=train1.index, y=train1.EMA_8, line=dict(color='blue', width=1), name="EMA8"),
        go.Scatter(x=train1.index, y=train1.EMA_14, line=dict(color='red', width=1), name="EMA14"),
        go.Scatter(x=train1.index, y=train1.EMA_50, line=dict(color='green', width=1), name="EMA50")]
        ).update_layout(
        yaxis_domain=[0.3, 1],
        yaxis2={"domain": [0, 0.20]},
        xaxis_rangeslider_visible=False,
        showlegend=False,
 )

fig_001.add_scatter(x=train1.index, y=train1['sells'], mode="markers",
                marker=dict(size=10, color="red"),
                name="sell")
fig_001.add_scatter(x=train1.index, y=train1['buys'], mode="markers",
                marker=dict(size=10, color="green"),
                name="buy")

fig_001.show()

In [30]:
train = BTCUSDT15m1['2018-01-01 00:00:00': '2019-01-01 00:00:00']
test = BTCUSDT15m1['2019-02-01 00:00:00': '2020-02-01 00:00:00']

In [31]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [32]:
class TradePro(Strategy):
    initsize = .05
    mysize = initsize
    def init(self):
        close = self.data.Close
        high = self.data.High
        low = self.data.Low
        self.stoch_k = self.I(ta.momentum.stochrsi_k, pd.Series(close))
        self.stoch_d = self.I(ta.momentum.stochrsi_d, pd.Series(close))
        self.EMA_8 = self.I(ta.trend.ema_indicator, pd.Series(close), window = 8)
        self.EMA_14 = self.I(ta.trend.ema_indicator, pd.Series(close), window = 14)
        self.EMA_50 = self.I(ta.trend.ema_indicator, pd.Series(close), window = 50)
        self.atr = self.I(ta.volatility.average_true_range, pd.Series(high), pd.Series(low), pd.Series(close))

    def next(self):
        price = self.data.Close
        if (crossover(self.stoch_k, self.stoch_d) and
            price > self.EMA_8 and
            self.EMA_8 > self.EMA_14 and
            self.EMA_14 > self.EMA_50):
                sl = 1000
                tp = price + self.atr*1.5
                self.buy(sl = sl, tp = tp, size = self.mysize)

In [ ]:
# s.loc['Sharpe Ratio'] = np.clip((s.loc['Return (Ann.) [%]'] - risk_free_rate) / (s.loc['Volatility (Ann.) [%]'] or np.nan), 0, np.inf)

In [33]:
bt1 = Backtest(train, TradePro, cash = 100000, exclusive_orders=True)

In [34]:
output1 = bt1.run()
output1

Start                     2018-01-01 12:15:00
End                       2019-01-01 00:00:00
Duration                    364 days 11:45:00
Exposure Time [%]                    8.367165
Equity Final [$]                 98533.397729
Equity Peak [$]                      100000.0
Return [%]                          -1.466602
Buy & Hold Return [%]               -71.87534
Return (Ann.) [%]                   -1.462625
Volatility (Ann.) [%]                1.169235
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.610111
Avg. Drawdown [%]                   -1.610111
Max. Drawdown Duration       41 days 08:15:00
Avg. Drawdown Duration       41 days 08:15:00
# Trades                                  119
Win Rate [%]                        63.865546
Best Trade [%]                       2.379447
Worst Trade [%]                    -12.117175
Avg. Trade [%]                    

In [35]:
#output1._trades.head(20)

In [58]:
bt2 = Backtest(test, TradePro, cash = 100000, exclusive_orders=True)

In [59]:
output2 = bt2.run()
output2

Start                     2019-02-01 00:00:00
End                       2020-02-01 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   11.748368
Equity Final [$]                100862.089208
Equity Peak [$]                 100863.479208
Return [%]                           0.862089
Buy & Hold Return [%]              172.965111
Return (Ann.) [%]                    0.874612
Volatility (Ann.) [%]                0.532794
Sharpe Ratio                         1.641557
Sortino Ratio                        2.163839
Calmar Ratio                         2.035241
Max. Drawdown [%]                   -0.429734
Avg. Drawdown [%]                   -0.028082
Max. Drawdown Duration      294 days 08:00:00
Avg. Drawdown Duration        6 days 14:31:00
# Trades                                  265
Win Rate [%]                        67.169811
Best Trade [%]                        1.59707
Worst Trade [%]                     -7.412735
Avg. Trade [%]                    

# VWAP SIGNAL

In [38]:
%%time
BTCUSDT15m['VWAP'] = fn.vwap(BTCUSDT15m.High, BTCUSDT15m.Low, BTCUSDT15m.Close, BTCUSDT15m.Volume)

CPU times: total: 31.2 ms
Wall time: 39.5 ms


In [39]:
BTCUSDT15m['VWAP_buy_signal'] = BTCUSDT15m['Close'] > BTCUSDT15m['VWAP']
BTCUSDT15m['VWAP_buy_signal'] = BTCUSDT15m['VWAP_buy_signal'].rolling(3).sum() >= 3
BTCUSDT15m['VWAP_buy_signal'] = BTCUSDT15m['VWAP_buy_signal'].astype(int)

# PIVOTS SIGNAL

In [40]:
pivots = fn.pivots(BTCUSDT15m['Open'], BTCUSDT15m['High'], BTCUSDT15m['Low'], BTCUSDT15m['Close'], anchor=15, method='fibonacci')

# SRSI SIGNAL

In [41]:
%%time
BTCUSDT15m = pd.concat([BTCUSDT15m, pta.stochrsi(BTCUSDT15m.Close, 14)], axis = 1)

CPU times: total: 46.9 ms
Wall time: 45.5 ms


In [42]:
BTCUSDT15m.rename(columns = {'STOCHRSIk_14_14_3_3':'K', 'STOCHRSId_14_14_3_3':'D'}, inplace = True)

In [43]:
BTCUSDT15m.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'VWAP_buy_signal',
       'K', 'D'],
      dtype='object')

In [44]:
BTCUSDT15m['SRSI_buy'] = BTCUSDT15m['K'] > BTCUSDT15m['D']
BTCUSDT15m['SRSI_sell'] = BTCUSDT15m['K'] < BTCUSDT15m['D']

BTCUSDT15m['SRSI_buy_signal'] = BTCUSDT15m['SRSI_buy'].rolling(2).sum() >= 2
BTCUSDT15m['SRSI_buy_signal'] = BTCUSDT15m['SRSI_buy_signal'].astype(int)
BTCUSDT15m['SRSI_sell_signal'] = BTCUSDT15m['SRSI_sell'].rolling(3).sum() >= 3

In [45]:
BTCUSDT15m['SRSI_sell_signal'] = BTCUSDT15m['SRSI_sell_signal'].map({True: 2 , False: 0})

In [46]:
BTCUSDT15m['SRSI_signal'] = BTCUSDT15m['SRSI_buy_signal']+BTCUSDT15m['SRSI_sell_signal']

In [47]:
BTCUSDT15m[['VWAP_buy_signal', 'SRSI_signal']]

,VWAP_buy_signal,SRSI_signal
Open Time,,
2018-01-01 00:00:00,0,0
2018-01-01 00:15:00,0,0
2018-01-01 00:30:00,0,0
2018-01-01 00:45:00,0,0
2018-01-01 01:00:00,0,0
...,...,...
2022-07-06 02:45:00,0,1
2022-07-06 03:00:00,0,0
2022-07-06 03:15:00,0,0


# TOTAL SIGNAL

In [48]:
BTCUSDT15m['buy'] = [1 if BTCUSDT15m['VWAP_buy_signal'][i] == 1 and BTCUSDT15m['SRSI_signal'][i] == 1 else 0 for i in range(len(BTCUSDT15m))]

In [49]:
Counter(BTCUSDT15m['buy'])

Counter({0: 132437, 1: 25166})

In [50]:
BTCUSDT15m['sell'] = [2 if BTCUSDT15m['SRSI_signal'][i] == 2 else 0 for i in range(len(BTCUSDT15m))]

In [51]:
Counter(BTCUSDT15m['sell'])

Counter({0: 114759, 2: 42844})

In [52]:
BTCUSDT15m['position'] = BTCUSDT15m['sell']+BTCUSDT15m['buy']

In [53]:
Counter(BTCUSDT15m['position'])

Counter({0: 89593, 2: 42844, 1: 25166})

In [54]:
def pointpos(x):
    if x['position']== 1:
        return x['High']+1e-3
    elif x['position']== 2:
        return x['Low']-1e-3
    else:
        return np.nan

BTCUSDT15m['pointpos'] = BTCUSDT15m.apply(lambda row: pointpos(row), axis=1)

In [55]:
t1 = BTCUSDT15m['2021-03-18 19:00:00': '2021-03-21 00:00:00']
train = BTCUSDT15m['2018-01-01 00:00:00': '2019-01-01 00:00:00']
test = BTCUSDT15m['2019-02-01 00:00:00': '2020-02-01 00:00:00']

In [56]:
%%time
pivs = fn.pivots(t1['Open'], t1['High'], t1['Low'], t1['Close'], anchor=15, method='fibonacci')

CPU times: total: 15.6 ms
Wall time: 11 ms


In [57]:
fig_001 = go.Figure(
    data=[
        go.Candlestick(
            x=t1.index,
            open=t1["Open"],
            high=t1["High"],
            low=t1["Low"],
            close=t1["Close"],
            name="OHLC",
        ),
        go.Scatter(x=t1.index, y=t1['K'], line=dict(color='blue', width=1.5),mode="lines", name="K", yaxis="y2"),
        go.Scatter(x=t1.index, y=t1['D'], line=dict(color='red', width=1.5),mode="lines", name="D", yaxis="y2"),
        go.Scatter(x=t1.index, y=t1.VWAP, line=dict(color='blue', width=1.5), name="VWAP"),
        go.Scatter(x=pivs.index, y=pivs.p, line=dict(color='red', width=1.5), name="P"),
        go.Scatter(x=pivs.index, y=pivs.s1, line=dict(color='orange', width=1), name="S1"),
        go.Scatter(x=pivs.index, y=pivs.s2, line=dict(color='orange', width=1), name="S2"),
        go.Scatter(x=pivs.index, y=pivs.r1, line=dict(color='orange', width=1), name="R1"),
        go.Scatter(x=pivs.index, y=pivs.r2, line=dict(color='orange', width=1), name="R2")]
        ).update_layout(
        yaxis_domain=[0.3, 1],
        yaxis2={"domain": [0, 0.20]},
        xaxis_rangeslider_visible=False,
        showlegend=False,
 )

fig_001.add_scatter(x=t1.index, y=t1['pointpos'], mode="markers",
                marker=dict(size=7, color="MediumPurple"),
                name="Signal")

fig_001

![](files/strategy.png)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=972d23a0-f342-484b-9096-244a1ebf7fc5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>